In [ ]:
!pip install tensorflow[and-cuda]

In [ ]:
#GW: I added this pip install
!pip install SpeechRecognition
!pip install google-cloud-texttospeech

In [ ]:
# Install Gradio
!pip install gradio


In [5]:
#1 RUN THIS FIRST FOR THE DEMO WITHOUT MY_MODEL.KERAS USING GEMMA2BINSTRUCT INSTEAD.

#start off running this for the gemma 2b instruct model
# import keras so it is defined for later use
import keras
import keras_nlp

# Load the saved model in .keras format
# Load the saved model in .keras format
#GW: Note that I don't have your model, so I'm loading a default one.
#GW: Need to get permissions via KaggleHub thus the login step.
#GW: gemma_lm = keras.models.load_model('my_model.keras')
import kagglehub
kagglehub.login()

In [ ]:
#2 FOR THE DEMO WITHOUT  MY_MODEL.KERAS 
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.summary()

## 1 FOR THE DEMO run this first to load the Fine-Tuned model

In [ ]:
#1 run this first to load the finetuned model
#start off running this
# import keras so it is defined for later use
import keras
import keras_nlp

# Load the saved model in .keras format
gemma_lm = keras.models.load_model('my_model.keras')


## 2 FOR THE DEMO run this after you load the model

In [ ]:
#2 FOR THE DEMO run this after you load the model
import datetime
import gradio as gr
import tensorflow as tfG
import keras
import speech_recognition as sr
from google.cloud import texttospeech

#GW: I moved the imports to the cell above

# Initialize the Text-to-Speech client
client = texttospeech.TextToSpeechClient()

# Initialize the Speech Recognition
recognizer = sr.Recognizer()

# Function to process Speech-to-Text
def speech_to_text(audio):
    try:
        print("sr.AudioFile start: ", datetime.datetime.now(), "audio=", type(audio), audio)
        with sr.AudioFile(audio) as source:
            print("sr.AudioFile start2: ", datetime.datetime.now(), "source=", type(source))
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            print("sr.AudioFile end: ", datetime.datetime.now())
            return text
    except sr.UnknownValueError:
        return "Sorry, I couldn't understand the audio. Please try again."
    except sr.RequestError as e:
        return f"Request error from Google STT service; {e}"

# Function to generate response from the chatbot model
def generate_response(model: keras.Model, question: str, max_length: int = 250) -> str:
    prompt_template = """ Role: You will use layman english to explain yourself. You will explain everything in simple terms using layman english.
    You are a helpful real estate assistant providing support to legal matters in Ontario, 
    Canada Region; always answer the questions based on the following instructions; given the question, 
    generate an answer in 5 to 6 sentences always. For every statement in the answer, provide supporting 
    legal document or law segments from Ontario, Canada region.
    Question:\n{question}\n\nAnswer:\n"""
    
    prompt = prompt_template.format(question=question)
    print("generate_response start: ", datetime.datetime.now(), "prompt-->", prompt,"<--")
    
    try:
        generated_response = model.generate(prompt, max_length=max_length)
        generated_text = generated_response.decode('utf-8') if isinstance(generated_response, bytes) else generated_response
        answer_text = generated_text[len(prompt):].strip()
        print("generate_response generated: ", datetime.datetime.now(), "answer text-->", answer_text, "<--")
        return answer_text
    except Exception as e:
        return f"⚠️ Error generating answer: {e}"

# Function to synthesize text using Google Text-to-Speech
def synthesize_text(text):
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Standard-C",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
    
    print("client synthesize_speech start: ", datetime.datetime.now())
    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )
    print("client synthesize_speech finished: ", datetime.datetime.now())
    audio_file_path = "output.mp3"
    with open(audio_file_path, "wb") as out:
        out.write(response.audio_content)
    return audio_file_path

# Combined function to handle chatbot response and TTS
def chatbot_with_tts_and_stt(audio):
    print("chatbot start: ", datetime.datetime.now())
    # Convert speech to text
    user_input = speech_to_text(audio)
    if not user_input.strip():
        return "Chatbot: I'm here to help! Please try speaking again.", None
    
    # Confirm the input text
    confirmation_message = f"Recognized text: '{user_input}'. Is this correct? If so, proceed. If not, try again."
    # Optionally, you can add a confirmation mechanism here
    
    # Generate chatbot response
    answer = generate_response(gemma_lm, user_input, max_length=250)
    
    # Synthesize the generated text into speech
    audio_file_path = synthesize_text(answer)
    
    print("chatbot finish: ", datetime.datetime.now())
    return answer, audio_file_path

# Create Gradio interface THIS LOOKS GOOD NEED REMOVE WATERMARK CHARACTER
interface = gr.Interface(
    fn=chatbot_with_tts_and_stt,
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs=[gr.Textbox(), gr.Audio(autoplay=True)],
    title="""
    <div style="display: flex; flex-direction: column; align-items: center; text-align: center; margin-bottom: 10px;">
        <img src="http://getmuggle.com/wp-content/uploads/2024/11/Logo-shrunk.png" style="height: 180px; margin-bottom: -10px;">
        <h1 style="margin: 0; font-size: 24px;">Real Estate Mate</h1>
    </div>
    """,
    description="Speak your question and get a spoken answer.",
    # theme='Zarkel/IBM_Carbon_Theme'
    theme='NoCrypt/miku'
    # theme='shivi/calm_seafoam'
    # theme='gstaff/xkcd'
    # theme='NoCrypt/miku'
    # theme='Base'
)

# Launch the interface
demo = interface.launch(share=True)


In [ ]:
interface.close()

In [ ]:
#if you want to change the model run this to clear the memory
import gc

del gemma_lm  # Delete the model
gc.collect()  # Run garbage collection to free up memory
